In [1]:
import os

In [2]:
#pwd

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/vikashinnamuri/Desktop/Chicken_disease_classification'

In [5]:
import tensorflow as tf

In [7]:
model=tf.keras.models.load_model("artifacts/training/model.h5")

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvalutionConfig:
    path_of_model: Path
    tranining_data: Path
    all_params:dict
    params_image_size:list
    params_batch_size:int

In [9]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml,create_directories,save_json

In [11]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        param_filepath=PARAMS_FILE_PATH):
        
        self.config= read_yaml(config_filepath)
        self.params=read_yaml(param_filepath)
        
        create_directories([self.config.artifacts_root],verbose=True)
        
    def get_validation_config(self)->EvalutionConfig:
        eval_config=   EvalutionConfig(
            path_of_model="artifacts/training/model.h5",
            tranining_data="artifacts/data_ingestion/Chicken-fecal-images",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
            
        )
        return eval_config

In [12]:
from urllib.parse import urlparse

In [17]:
class Evaluation:
    def __init__(self, config: EvalutionConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.tranining_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)

    
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

In [18]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()

except Exception as e:
   raise e

[2024-08-30 10:11:08,069: INFO: common:  yaml file:config/config.yaml successfully loaded]
[2024-08-30 10:11:08,073: INFO: common:  yaml file:params.yaml successfully loaded]
[2024-08-30 10:11:08,074: INFO: common: Created directories at: artifacts]
Found 116 images belonging to 2 classes.
8/8 [==============================] - 11s 1s/step - loss: 0.3936 - accuracy: 0.9483
[2024-08-30 10:11:19,227: INFO: common: json file saved at:scores.json]
